# Python Programming Language for Data Analysis with Pandas Library


**In this workshop we will analyze the data set with information on return and Morningstar Sustainalytics rating of 2000+ ETF from 2015 to 2021 using the `pandas` library.** 

Python programming language has numerous functions. These functions are organized into modules. Modules are python scripts that contain Python objects. In order to use the objects inside these modules (or commonly known as libraries), we must first import either the entire module or specific objects in it. 

**`pandas` (Python for Data Analysis)** is a popular python library for data analysis. We will use this library extensively in this workshop. This library is automatically installed during Anaconda installation. It is also installed and available on Google Colab or UofT Jupyter Hub. 

Other popular python libraries are **`NumPy` (Numerical Python)** and **`matplotlib`** used for scientific computing and visualization, respectively. We will use these libraries too. 

In [1]:
# import required libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
# read select columns only
col_list=['Name', 'Ticker', 'Base Currency', 'Global Broad Category Group',
          'Fund Size Base Currency', 'Domicile', 'month',
          'Monthly Return  Base Currency', 'Total Market Value(Net)  Portfolio Currency', 
          'asset_class', 'Morningstar Rating Overall', 'Carbon Risk Classification',
          'Portfolio Environmental Risk Score', 'Portfolio Governance Risk Score',
          'Portfolio Social Risk Score', 'Currency Code', 'Exchange Rate USD']
etf = pd.read_csv('ETF_Data_final.csv', sep=",", header=0, index_col=None, usecols=col_list)

In [3]:
# change column names
etf.columns = ['Name', 'Ticker', 'Base_Currency', 'Global_Broad_Category_Group',
               'Fund_Size_Base_Currency', 'Domicile', 'Month',
               'Monthly_Return_Base_Currency', 'Total_Market_Value_Portfolio_Currency', 
               'Asset_Class', 'Morningstar_Rating_Overall', 'Carbon_Risk_Classification',
               'Portfolio_Environmental_Risk_Score', 'Portfolio_Governance_Risk_Score',
               'Portfolio_Social_Risk_Score', 'Currency_Code', 'Exchange_Rate_USD']
etf.head()

,Name,Ticker,Base_Currency,Global_Broad_Category_Group,Fund_Size_Base_Currency,Domicile,Month,Monthly_Return_Base_Currency,Total_Market_Value_Portfolio_Currency,Asset_Class,Morningstar_Rating_Overall,Carbon_Risk_Classification,Portfolio_Environmental_Risk_Score,Portfolio_Governance_Risk_Score,Portfolio_Social_Risk_Score,Currency_Code,Exchange_Rate_USD
0,AAF First Priority CLO Bond ETF,AAA,US Dollar,Fixed Income,10002701.0,United States,2020-10-01,0.00281,9563230.0,bond,NaN,NaN,NaN,NaN,NaN,USD,1.0
1,AAF First Priority CLO Bond ETF,AAA,US Dollar,Fixed Income,10002701.0,United States,2020-11-01,0.61405,9615667.0,bond,NaN,NaN,NaN,NaN,NaN,USD,1.0
2,AAF First Priority CLO Bond ETF,AAA,US Dollar,Fixed Income,10002701.0,United States,2020-12-01,0.27885,9633997.0,bond,NaN,NaN,NaN,NaN,NaN,USD,1.0
3,AAF First Priority CLO Bond ETF,AAA,US Dollar,Fixed Income,10002701.0,United States,2021-01-01,0.43888,9173441.0,bond,NaN,NaN,NaN,NaN,NaN,USD,1.0
4,AAF First Priority CLO Bond ETF,AAA,US Dollar,Fixed Income,10002701.0,United States,2021-02-01,-0.13515,9609404.0,bond,NaN,NaN,NaN,NaN,NaN,USD,1.0


In [4]:
# convert to datetime object and create month and year columns
etf['date'] = pd.to_datetime(etf['Month'],format='%Y-%m-%d', errors='coerce')
etf['month'] = etf['date'].dt.month
etf['year'] = etf['date'].dt.year

# convert to USD currency
etf['monthly_return_USD'] = etf['Monthly_Return_Base_Currency']*etf['Exchange_Rate_USD']
etf['fund_size_USD'] = etf['Fund_Size_Base_Currency']*etf['Exchange_Rate_USD']

# convert to USD currency and to millions
etf['total_market_value_USD'] = etf['Total_Market_Value_Portfolio_Currency']*etf['Exchange_Rate_USD']
etf['total_mkt_val_net_USD_millions'] = etf['total_market_value_USD']/1000000

# create primary key based on ticker and currency code
etf['primary_key'] = etf['Ticker'] + etf['Currency_Code']

In [5]:
# drop any rows that have missing value in the returns or market value columns
etf.dropna(subset=['fund_size_USD', 'monthly_return_USD','total_mkt_val_net_USD_millions'], inplace=True)
etf.shape

(133999, 25)

In [6]:
# drop any columns with duplicate data
dup_col = ['Name', 'Ticker', 'Base_Currency', 'Fund_Size_Base_Currency', 'Month', 'Monthly_Return_Base_Currency',
          'Total_Market_Value_Portfolio_Currency', 'Currency_Code', 'Exchange_Rate_USD', 'date', 'total_market_value_USD']
other_col = ['fund_size_USD', 'total_mkt_val_net_USD_millions']
etf.drop(dup_col+other_col, axis=1, inplace=True)

In [7]:
etf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133999 entries, 0 to 140228
Data columns (total 12 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Global_Broad_Category_Group         133999 non-null  object 
 1   Domicile                            133999 non-null  object 
 2   Asset_Class                         133999 non-null  object 
 3   Morningstar_Rating_Overall          55007 non-null   float64
 4   Carbon_Risk_Classification          45990 non-null   object 
 5   Portfolio_Environmental_Risk_Score  61783 non-null   float64
 6   Portfolio_Governance_Risk_Score     61783 non-null   float64
 7   Portfolio_Social_Risk_Score         61783 non-null   float64
 8   month                               133999 non-null  int64  
 9   year                                133999 non-null  int64  
 10  monthly_return_USD                  133999 non-null  float64
 11  primary_key               

In [8]:
etf['Morningstar_Rating_Overall'].unique()

array([nan,  2.,  3.,  4.,  1.,  5.])

In [9]:
etf['Carbon_Risk_Classification'].unique()

array([nan, 'Medium Risk', 'Low Risk', 'Negligible Risk', 'High Risk'],
      dtype=object)

In [10]:
risk_scores = ['Portfolio_Environmental_Risk_Score', 'Portfolio_Governance_Risk_Score', 'Portfolio_Social_Risk_Score']
for i in risk_scores:
    print(f'\n{i}')
    print(etf[i].agg(['min', 'mean', 'max', 'median']))


Portfolio_Environmental_Risk_Score
min        0.000000
mean      16.984573
max       66.590000
median     5.370000
Name: Portfolio_Environmental_Risk_Score, dtype: float64

Portfolio_Governance_Risk_Score
min        0.000000
mean      18.545082
max       67.910000
median     7.840000
Name: Portfolio_Governance_Risk_Score, dtype: float64

Portfolio_Social_Risk_Score
min        0.000000
mean      19.931538
max       65.550000
median    10.240000
Name: Portfolio_Social_Risk_Score, dtype: float64


In [11]:
# convert missing rating to its own category
etf['Morningstar_Rating_Overall'].fillna(6, inplace=True)
etf['Carbon_Risk_Classification'].fillna('Unknown Risk', inplace=True)

In [12]:
# replace risk_score with median value
risk_scores = ['Portfolio_Environmental_Risk_Score', 'Portfolio_Governance_Risk_Score', 'Portfolio_Social_Risk_Score']
for i in risk_scores:
    etf[i].fillna(etf[i].median(), inplace=True)

In [13]:
etf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133999 entries, 0 to 140228
Data columns (total 12 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Global_Broad_Category_Group         133999 non-null  object 
 1   Domicile                            133999 non-null  object 
 2   Asset_Class                         133999 non-null  object 
 3   Morningstar_Rating_Overall          133999 non-null  float64
 4   Carbon_Risk_Classification          133999 non-null  object 
 5   Portfolio_Environmental_Risk_Score  133999 non-null  float64
 6   Portfolio_Governance_Risk_Score     133999 non-null  float64
 7   Portfolio_Social_Risk_Score         133999 non-null  float64
 8   month                               133999 non-null  int64  
 9   year                                133999 non-null  int64  
 10  monthly_return_USD                  133999 non-null  float64
 11  primary_key               

## 5. Dealing with Categorical Data

In [14]:
cat_date = etf[['month','year']].apply(lambda x : pd.Categorical(x))
cat_date.head()

,month,year
0,10,2020
1,11,2020
2,12,2020
3,1,2021
4,2,2021


In [15]:
cat_date.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133999 entries, 0 to 140228
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype   
---  ------  --------------   -----   
 0   month   133999 non-null  category
 1   year    133999 non-null  category
dtypes: category(2)
memory usage: 1.3 MB


In [16]:
etf[['month','year']] = etf[['month','year']].apply(lambda x : pd.Categorical(x))
etf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133999 entries, 0 to 140228
Data columns (total 12 columns):
 #   Column                              Non-Null Count   Dtype   
---  ------                              --------------   -----   
 0   Global_Broad_Category_Group         133999 non-null  object  
 1   Domicile                            133999 non-null  object  
 2   Asset_Class                         133999 non-null  object  
 3   Morningstar_Rating_Overall          133999 non-null  float64 
 4   Carbon_Risk_Classification          133999 non-null  object  
 5   Portfolio_Environmental_Risk_Score  133999 non-null  float64 
 6   Portfolio_Governance_Risk_Score     133999 non-null  float64 
 7   Portfolio_Social_Risk_Score         133999 non-null  float64 
 8   month                               133999 non-null  category
 9   year                                133999 non-null  category
 10  monthly_return_USD                  133999 non-null  float64 
 11  primary_key  

In [17]:
cat_cols = etf.select_dtypes(include=['object', 'category']).columns
cat_cols

Index(['Global_Broad_Category_Group', 'Domicile', 'Asset_Class',
       'Carbon_Risk_Classification', 'month', 'year', 'primary_key'],
      dtype='object')

### 5a. Creating Dummy Variables

What are the ways to convert categorical feature into numeric values? There are many. One popular method is to create a new column for each categorical variable and fill in value 1 for the specified routename column if the row belongs to that routename. This process is also called creating dummy variable.

Pandas has an easy way to create dummy variables using the `get_dummies` method. Let's apply that on a toy dataset to see what transformation is taking place.

In [18]:
# create toy data set with categorica and numeric features
test_data = pd.DataFrame( [['apple', 2], ['orange', 5]], columns=['fruits', 'quantity'])
test_data.head()

,fruits,quantity
0,apple,2
1,orange,5


In [19]:
# create dummy variables for a categorical column
pd.get_dummies(test_data['fruits'])

,apple,orange
0,1,0
1,0,1


In [20]:
# combine the dummy variables with remaining numeric column of the original data
pd.concat([pd.get_dummies(test_data['fruits']), test_data['quantity']], axis=1)

,apple,orange,quantity
0,1,0,2
1,0,1,5


### 5b. One Hot Encoding

The above transformation can also be done via OneHotEncoder function available through the preprocessing submodule of sklearn. This method is easier to use if you have many categorical columns. Here, we will see an example on our toy data set.

In [21]:
from sklearn.preprocessing import OneHotEncoder

In [22]:
# 1. create an instance of the one hot encoder object
ohe = OneHotEncoder()

# 2. fit the data to the one hot encoder instance
ohe.fit_transform(test_data['fruits'].values.reshape(-1,1)) # require the input data to be 2-dimensional

<2x2 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

The resulting object is a sparse matrix, which cannot be combined to a `dataframe`. Therefore, we must first convert it to a numpy `array` for which there is a `toarray` method provided.

In [23]:
# 3. convert the transformed data to an numpy array 
dummy_fruits = ohe.fit_transform(test_data['fruits'].values.reshape(-1,1)).toarray()
dummy_fruits

array([[1., 0.],
       [0., 1.]])

Numpy `array` can be easily converted to a pandas `dataframe`.

In [24]:
# 4. transform the numpy array to dataframe
pd.DataFrame(dummy_fruits)

,0,1
0,1.0,0.0
1,0.0,1.0


Note that we are missing the column names, which can make it difficult to know which category the column belongs to. This can be easily retrieved using the `categories_` method of the `OneHotEncoder` object. 

In [25]:
# while converting to dataframe add column names as well
pd.DataFrame(dummy_fruits, columns=ohe.categories_)

,apple,orange
0,1.0,0.0
1,0.0,1.0


You could also use the `get_feature_names_out` method, which can take a string to add to each column name.

In [26]:
ohe.get_feature_names_out(['fruits'])

array(['fruits_apple', 'fruits_orange'], dtype=object)

Now, let's try this on our date that have categorical values.

In [27]:
for i in cat_cols:
    print(f'\n{i}')
    n_uniq = etf[i].nunique()
    print(n_uniq)
    if n_uniq < 10:
        print(etf[i].value_counts())


Global_Broad_Category_Group
7
Equity           101599
Fixed Income      14674
Miscellaneous     13054
Alternative        2369
Commodities        1670
Allocation          445
Convertibles        188
Name: Global_Broad_Category_Group, dtype: int64

Domicile
3
United States     120118
Canada              9029
United Kingdom      4852
Name: Domicile, dtype: int64

Asset_Class
4
equity         113018
bond            14973
real_estate      3882
commodity        2126
Name: Asset_Class, dtype: int64

Carbon_Risk_Classification
5
Unknown Risk       88009
Low Risk           24391
Medium Risk        20501
High Risk            879
Negligible Risk      219
Name: Carbon_Risk_Classification, dtype: int64

month
12

year
7
2021    29384
2020    25313
2019    22748
2018    16647
2017    14821
2016    13402
2015    11684
Name: year, dtype: int64

primary_key
2572


In [28]:
# transform all categorical columns using one hot encoder except primary key
ohe = OneHotEncoder()
cat_data = ohe.fit_transform(etf[cat_cols[:-1]]).toarray() # do not include primary key
cat_data = pd.DataFrame(cat_data, columns=ohe.get_feature_names_out(cat_cols[:-1]))
cat_data.index = etf.index
cat_data.shape

(133999, 38)

In [29]:
# drop original categorical columns and add the transformed ones instead
etf_transformed = etf.drop(cat_cols, axis=1)
etf_transformed.shape

(133999, 5)

In [30]:
# combine numeric features with transformed categorical features
etf_transformed = pd.concat([etf_transformed,cat_data], axis=1)
etf_transformed.shape

(133999, 43)

### 5c. Encoding Labels

In [31]:
from sklearn.preprocessing import LabelEncoder

In [32]:
le = LabelEncoder()
le.fit_transform(etf['Global_Broad_Category_Group'])

array([5, 5, 5, ..., 5, 5, 5])

In [33]:
le.fit_transform(etf['Global_Broad_Category_Group'])

array([5, 5, 5, ..., 5, 5, 5])

## 6. Data Modelling with Scikit-Learn

There are many ways to model the data and the best way really depends on the problem you are trying to solve. Let's say for example, we want to know if it is possible to use the information available in our transformed data to predict the monthly return.

Prior to modelling the data, the data must be clean and any item in the data must be numeric. This is because machine learning models or statistical models do not take data that have textual values or missing values. So such data must be processed and transformed to some reasonable numeric representation before they are used in modelling. 

Since we have already done the data cleaning and processing part, we will begin by separating the variable that will be predicted from the variables that will be use to predict. The former is commonly refered to as target (y) and the latter as features (X).

In [34]:
X = etf_transformed.drop(['monthly_return_USD'], axis=1)
y = etf_transformed['monthly_return_USD']

In [35]:
X.head()

,Morningstar_Rating_Overall,Portfolio_Environmental_Risk_Score,Portfolio_Governance_Risk_Score,Portfolio_Social_Risk_Score,Global_Broad_Category_Group_Allocation,Global_Broad_Category_Group_Alternative,Global_Broad_Category_Group_Commodities,Global_Broad_Category_Group_Convertibles,Global_Broad_Category_Group_Equity,Global_Broad_Category_Group_Fixed Income,...,month_10,month_11,month_12,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021
0,6.0,5.37,7.84,10.24,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,6.0,5.37,7.84,10.24,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,6.0,5.37,7.84,10.24,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,6.0,5.37,7.84,10.24,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,6.0,5.37,7.84,10.24,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [36]:
y[:5]

0    0.00281
1    0.61405
2    0.27885
3    0.43888
4   -0.13515
Name: monthly_return_USD, dtype: float64

Next, we will split the feature data into training and test set. This is while we use data to train a machine learning model, its performance should be reported on a data that the model has never seen before. This ensures that the model is able to generalize i.e. it has not just learned the training data very well but also some patterns that can help predict future data points. This is very important if we want to use the model in the real world.

Usually a 70-30 or 80-20 split is recommended. In our case, we will keep 30% of the data for testing and train the model with remaining 70% of the data. Let's import the `train_test_split` function available via the `model_selection` submodule of `sklearn` library. This function takes the feature and target data and give us the desired splits of the data. 

In [37]:
from sklearn.model_selection import train_test_split as tts

In [38]:
X_train, X_test, y_train, y_test = tts(X,              # feature data
                                       y,              # target data 
                                       test_size=.30,  # size of the test set
                                       random_state=42)# set a random number to get the exact split next time 

In [39]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(93799, 42) (40200, 42) (93799,) (40200,)


You can see that 20% of the data for both features and target are now held-out as the test set. 

### 6a. Linear Regression
Linear Regression is the most used statistical algorithm. It examines a linear relationship between two or more variables. Linear Regression provides a good background in understanding a whole class of linear models known as Generalized Linear Models (GLM). It also helps understand statistical terms such as cost functions, coefficients and optimization. 

The overall objective of Linear Regression is to predict a straight line through the data, such that the vertical distance of each data point is minimal from that line.

Let's now import the `Linear_Regression` model from the `linear_model` submodule of `sklearn`, which we will use to fit our data.

In [40]:
from sklearn.linear_model import LinearRegression

Most of the functions in sklearn can be used in the same way:

1. Create an instance of the object in use.
2. Use `fit` or `fit_transform` method to fit or transform the data as needed. 

In [41]:
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

Now that the data has been fit to the linear model, some model property information will now be available in the `model` object. One of these properties is the `score` method, which returns the coefficient of determination of the prediction, also known as R squared. It is the proportion of the variation in the dependent variable that is predictable form the independent variable. We can input the training data to get this score.

Other properties of interest are the intercept and the coefficients of the linear regression model. We can get these information with the `intercept_` and `coef_` methods respectively.

In [42]:
# return the coefficient of determination of the prediction
model.score(X_train, y_train)

0.002265433163255448

In [43]:
model.intercept_

3.034094003604393

In [44]:
model.coef_

array([-0.02835847,  0.04473452,  0.01876206, -0.0681953 ,  0.21158945,
        1.36047047, -2.92853603,  0.11323771,  1.8329775 , -0.91875368,
        0.32901458,  3.75975934, -1.32739484, -2.43236449,  0.41342607,
        2.25771068, -1.28035563, -1.39078112, -0.84891828,  0.48299489,
       -0.45748469,  0.36232662,  0.46108145,  0.75855728, -0.92645283,
       -2.49362126,  2.82413288, -0.47930915,  0.31343101,  0.44857785,
       -0.86410168, -2.1507675 , -0.42698793,  2.73204843,  0.26449291,
       -1.59002016,  1.59605138,  0.51663461, -1.96972057,  0.50276752,
        0.90567084,  0.03861637])

In [45]:
coefs = {}
for i,j in zip(X_train.columns, model.coef_[1:]):
    coefs[i] = j
pd.DataFrame(coefs.items(), columns=['feature','coef']).sort_values(by='coef', key=abs, ascending=False).round(4)

,feature,coef
10,Global_Broad_Category_Group_Miscellaneous,3.7598
5,Global_Broad_Category_Group_Alternative,-2.9285
25,month_3,2.8241
32,month_10,2.7320
24,month_2,-2.4936
12,Domicile_United Kingdom,-2.4324
14,Asset_Class_bond,2.2577
30,month_8,-2.1508
37,year_2017,-1.9697
7,Global_Broad_Category_Group_Convertibles,1.8330


R Squared value only measures the fit of training data to the model. How well will this model perform on an unseen test data is the next step of evaluation. Regression models often use the mean squared error metric to evaluate the performance on an unseen data. To calculate this we can use `mean_squared_error` function available throuhg the `metrics` submodule of `sklearn`. The function takes the model prediction on a given data and the actual target value for that dataset. Therefore, we first need to generate predictions from our model on the test set using the `predict` method.

In [46]:
from sklearn.metrics import mean_squared_error as mse

In [47]:
y_pred_test = model.predict(X_test)
mse_lrmodel = mse(y_pred_test, y_test)
print(mse_lrmodel)

863.5539077021615


How do we know this is a good enough value?

In machine learning, we usually have benchmark model against which we can test the performance. In this case we only have one model, so we can create another model and see which one performs better. 

We can repeat what we did earlier on another model or we can create a for-loop such that the exact same operation goes through all the models in a list. This latter is obviously better as we do not have to write the same code over and over again. It is also easier from readability perspective.

### 6b. Decision Trees

In [48]:
from sklearn.tree import DecisionTreeRegressor

In [49]:
# initialize an empty list to add sklearn model objects
models = []

# add the sklearn model objects to the list one by one
# while adding the model also give it a name so put the name and model in a tuple
models.append(('LR', LinearRegression())) 
models.append(('DT', DecisionTreeRegressor())) # Ensemble method - collection of many decision trees
models

[('LR', LinearRegression()), ('DT', DecisionTreeRegressor())]

In [50]:
scores = {}
for name, model in models:
    print(model)
    model.fit(X_train, y_train)
    y_pred_test = model.predict(X_test)
    mse_score = mse(y_pred_test, y_test)
    scores[name] = mse_score
scores

LinearRegression()
DecisionTreeRegressor()


{'LR': 863.5539077021615, 'DT': 942.1607929570563}

The Decision Tree Regression model has higher mean squared error than the Linear Regression model and therefore, Linear Regression model is better than the Decision Tree model.

__Note:__ It might also be a good idea to store the fitted model, so that once you can explore more details of the model rather than just the scores. See [here](https://scikit-learn.org/stable/modules/tree.html#tree-regression) to learn more about decision tree model properties and sklearn features available to explore the model details.

Recall earlier we did not use the primary_key column to in our model because there were many categories. If we repeat the above modelling process including the information in this column, are we likely to make a better prediction? Let's check that now.

In [51]:
# transform all categorical columns using one hot encoder including primary key
ohe = OneHotEncoder()
cat_data = ohe.fit_transform(etf[cat_cols]).toarray() # do not include primary key
cat_data = pd.DataFrame(cat_data, columns=ohe.get_feature_names_out(cat_cols))
cat_data.index = etf.index

# drop original categorical columns and add the transformed ones instead
etf_transformed = etf.drop(cat_cols, axis=1)

# combine numeric features with transformed categorical features
etf_transformed = pd.concat([etf_transformed,cat_data], axis=1)
etf_transformed.shape

(133999, 2615)

In [52]:
# create feature and target variables
X = etf_transformed.drop(['monthly_return_USD'], axis=1)
y = etf_transformed['monthly_return_USD']

# split data into training and test set
X_train, X_test, y_train, y_test = tts(X, y, test_size=.30, random_state=42)

In [53]:
# initialize an empty list to add sklearn model objects
models = []

# add the sklearn model objects to the list one by one
# while adding the model also give it a name so put the name and model in a tuple
models.append(('LR', LinearRegression())) 
models.append(('DT', DecisionTreeRegressor())) # Ensemble method - collection of many decision trees
models

scores = {}
for name, model in models:
    print(model)
    model.fit(X_train, y_train)
    y_pred_test = model.predict(X_test)
    mse_score = mse(y_pred_test, y_test)
    scores[name] = mse_score
scores

LinearRegression()
DecisionTreeRegressor()


{'LR': 553594206705754.5, 'DT': 897.1922295122931}

Adding information in the primary_key column leads to increase in the error of the linear regression model but decreases the same for decision tree model. This is common behavior when comparing a linear and non-linear models with large number of variables. 